# 🧬 CHDgene Metadata Extraction Notebook
This notebook fetches, extracts, and saves gene-disease-phenotype-reference metadata from the CHDgene database.

## 1. Setup and Constants

In [14]:
import requests
import pandas as pd

BASE_URL = "https://chdgene.victorchang.edu.au/"

def fetch_gene_list():
    """
    Fetches the list of genes available in the CHDgene database.
    
    Returns:
        list: A list of dictionaries, each representing a gene summary with gene ID and symbol.
    """
    """Fetch all gene summaries from CHDgene API."""
    res = requests.get(f"{BASE_URL}/genes")
    res.raise_for_status()
    return res.json()

def fetch_gene_detail(gene_id):
    """Fetch full gene detail (metadata, phenotypes, references) for one gene."""
    res = requests.get(f"{BASE_URL}/gene/{gene_id}")
    res.raise_for_status()
    return res.json()

def extract_gene_metadata(detail):
    """Flatten gene-level metadata and references into a list of rows (one per reference)."""
    gene_id = detail.get("id", "")
    symbol = detail.get("symbol", "")
    name = detail.get("name", "")
    synonyms = "; ".join(detail.get("synonyms", []))
    disease = detail.get("disease", "")
    phenotypes = "; ".join(detail.get("chdPhenotypes", []))
    references = detail.get("selectedReferences", [])

    rows = []
    for ref in references:
        citation = ref.get("citation", "")
        doi = ref.get("doi", "")
        doi_link = f"https://doi.org/{doi}" if doi else ""
        scihub_link = f"https://sci-hub.ru/{doi}" if doi else ""

        rows.append({
            "gene_id": gene_id,
            "symbol": symbol,
            "gene_name": name,
            "synonyms": synonyms,
            "disease": disease,
            "phenotypes": phenotypes,
            "reference_text": citation,
            "doi_link": doi_link,
            "scihub_link": scihub_link
        })
    return rows

def extract_all_genes_to_dataframe():
    """High-level function to extract metadata for all genes into a DataFrame."""
    gene_list = fetch_gene_list()
    all_rows = []

    for gene in gene_list:
        gene_id = gene["id"]
        print(f"📥 Fetching {gene_id}")
        detail = fetch_gene_detail(gene_id)
        rows = extract_gene_metadata(detail)
        all_rows.extend(rows)

    return pd.DataFrame(all_rows)

def save_metadata_to_csv(filepath="CHDgene_Metadata_Simplified.csv"):
    """Extracts metadata and saves to CSV."""
    df = extract_all_genes_to_dataframe()
    df.to_csv(filepath, index=False)
    print(f"✅ Saved {len(df)} entries to {filepath}")


## 6. Exploratory Inspection

In [6]:
gene_list = extract_gene_list()
print(f"🔍 Found {len(gene_list)} genes.")

🔍 Found 189 genes.


In [43]:
def main():
    gene_list = extract_gene_list()
    print(f"🔍 Found {len(gene_list)} genes.")
    all_rows = []

    for i, (gene_id, symbol) in enumerate(gene_list):
        try:
            print(f"📄 Extracting {symbol} ({i+1}/{len(gene_list)})")
            gene_data = get_gene_metadata_with_merged_references(gene_id, symbol)

            refs = gene_data.get("reference_texts", [])
            dois = gene_data.get("doi_links", [])
            sci_links = gene_data.get("scihub_links", [])

            for j in range(len(refs)):
                row = {
                    "gene_id": gene_data["gene_id"],
                    "symbol": gene_data["symbol"],
                    "gene_name": gene_data["gene_name"],
                    "disease": gene_data["disease"],
                    "phenotypes": gene_data["phenotypes"],
                    "reference_text": refs[j] if j < len(refs) else "",
                    "doi_link": dois[j] if j < len(dois) else "",
                    "scihub_link": sci_links[j] if j < len(sci_links) else ""
                }
                all_rows.append(row)

            time.sleep(0.5)

        except Exception as e:
            print(f"❌ Error processing {symbol}: {e}")
            continue

    df = pd.DataFrame(all_rows)
    df.to_csv("CHDgene_Metadata_Exploded.csv", index=False)
    print(f"✅ Saved exploded metadata with one row per reference to: CHDgene_Metadata_Exploded.csv")

if __name__ == "__main__":
     main()


🔍 Found 189 genes.
📄 Extracting ABL1 (1/189)
📄 Extracting ACTC1 (2/189)
📄 Extracting ACVR1 (3/189)
📄 Extracting ACVR2B (4/189)
📄 Extracting ADAMTS10 (5/189)
📄 Extracting AFF4 (6/189)
📄 Extracting ANKRD11 (7/189)
📄 Extracting ARID1A (8/189)
📄 Extracting ARID1B (9/189)
📄 Extracting B3GAT3 (10/189)
📄 Extracting BCOR (11/189)
📄 Extracting BMPR2 (12/189)
📄 Extracting BRAF (13/189)
📄 Extracting CDK13 (14/189)
📄 Extracting CFC1 (15/189)
📄 Extracting CHD4 (16/189)
📄 Extracting CHD7 (17/189)
📄 Extracting CHST14 (18/189)
📄 Extracting CITED2 (19/189)
📄 Extracting CREBBP (20/189)
📄 Extracting CRELD1 (21/189)
📄 Extracting DLL4 (22/189)
📄 Extracting DNAH11 (23/189)
📄 Extracting DNAH5 (24/189)
📄 Extracting DOCK6 (25/189)
📄 Extracting EFTUD2 (26/189)
📄 Extracting EHMT1 (27/189)
📄 Extracting ELN (28/189)
📄 Extracting EP300 (29/189)
📄 Extracting ESCO2 (30/189)
📄 Extracting EVC (31/189)
📄 Extracting EVC2 (32/189)
📄 Extracting FBN1 (33/189)
📄 Extracting FGFR2 (34/189)
📄 Extracting FLNA (35/189)
📄 Extracti

## 7. Miscellaneous

In [7]:
gene_list

[('25', 'ABL1'),
 ('70', 'ACTC1'),
 ('90', 'ACVR1'),
 ('93', 'ACVR2B'),
 ('81794', 'ADAMTS10'),
 ('27125', 'AFF4'),
 ('29123', 'ANKRD11'),
 ('8289', 'ARID1A'),
 ('57492', 'ARID1B'),
 ('26229', 'B3GAT3'),
 ('54880', 'BCOR'),
 ('659', 'BMPR2'),
 ('673', 'BRAF'),
 ('8621', 'CDK13'),
 ('55997', 'CFC1'),
 ('1108', 'CHD4'),
 ('55636', 'CHD7'),
 ('113189', 'CHST14'),
 ('10370', 'CITED2'),
 ('1387', 'CREBBP'),
 ('78987', 'CRELD1'),
 ('54567', 'DLL4'),
 ('8701', 'DNAH11'),
 ('1767', 'DNAH5'),
 ('57572', 'DOCK6'),
 ('9343', 'EFTUD2'),
 ('79813', 'EHMT1'),
 ('2006', 'ELN'),
 ('2033', 'EP300'),
 ('157570', 'ESCO2'),
 ('2121', 'EVC'),
 ('132884', 'EVC2'),
 ('2200', 'FBN1'),
 ('2263', 'FGFR2'),
 ('2316', 'FLNA'),
 ('2324', 'FLT4'),
 ('2296', 'FOXC1'),
 ('2303', 'FOXC2'),
 ('8928', 'FOXH1'),
 ('27086', 'FOXP1'),
 ('2626', 'GATA4'),
 ('140628', 'GATA5'),
 ('2627', 'GATA6'),
 ('2657', 'GDF1'),
 ('2697', 'GJA1'),
 ('2737', 'GLI3'),
 ('2719', 'GPC3'),
 ('23498', 'HAAO'),
 ('9421', 'HAND1'),
 ('9464', 'HA

In [11]:
entries = extract_metadata_and_references('25', 'ABL1')

None
None


In [17]:
res=requests.get(f"{BASE_URL}/gene/25")
res.raise_for_status()

In [36]:
def get_gene_metadata_v2(gene_id, symbol):
    """Robustly extract gene metadata from a CHDgene gene detail page using .row-based layout."""
    url = f"{BASE_URL}/gene/{gene_id}"
    res = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")

    gene_name = soup.find("h1").text.strip().split()[0]
    disease = ""
    phenotypes = []
     # --- Extract gene info ---
    rows = soup.find_all("div", class_="row")
    for row in rows:
        label_div = row.find("div", class_="col-md-3")
        value_div = row.find("div", class_="col-md-9")

        if not label_div or not value_div:
            continue

        label = label_div.get_text(strip=True)
        if label == "Disease":
            items = value_div.find_all("li")
            disease = "; ".join(li.get_text(strip=True) for li in items) if items else value_div.get_text(strip=True)

        elif label == "CHD Phenotype":
            items = value_div.find_all("li")
            phenotypes = [li.get_text(strip=True) for li in items]
            
    # --- Extract selected references ---
    references = []
    ref_header = soup.find("h3", string=lambda x: x and "Selected References" in x)
    if ref_header:
        ref_list = ref_header.find_next("ol")
        if ref_list:
            for li in ref_list.find_all("li"):
                citation = li.get_text(separator=" ", strip=True)
                doi = ""
                for s in li.find_all("strong"):
                    if "DOI" in s.text:
                        a_tag = s.find_next("a")
                        if a_tag and "doi.org" in a_tag["href"]:
                            doi = a_tag["href"].split("doi.org/")[-1]
                        elif a_tag:
                            doi = a_tag.text.strip()
                        break
                references.append({
                    "gene_id": gene_id,
                    "symbol": symbol,
                    "gene_name": gene_name,
                    "disease": disease,
                    "phenotypes": "; ".join(phenotypes),
                    "reference_text": citation,
                    "doi_link": f"https://doi.org/{doi}" if doi else "",
                    "scihub_link": f"https://sci-hub.ru/{doi}" if doi else ""
                })

    return references

In [37]:
get_gene_metadata_v2('70','ACTC1')

[{'gene_id': '70',
  'symbol': 'ACTC1',
  'gene_name': 'ACTC1',
  'disease': 'Atrial septal defect 5(AD)',
  'phenotypes': 'Atrial septal defect; Ventricular septal defect',
  'reference_text': 'Olson, T. M., Doan, T. P., Kishimoto, N. Y., Whitby, F. G., Ackerman, M. J., & Fananapazir, L. (2000). Inherited and de novo Mutations in the Cardiac Actin Gene Cause Hypertrophic Cardiomyopathy. Journal of Molecular and Cellular Cardiology, 32(9), 1687â€“1694. https://doi.org/10.1006/jmcc.2000.1204 DOI : 10.1006/jmcc.2000.1204',
  'doi_link': 'https://doi.org/10.1006/jmcc.2000.1204',
  'scihub_link': 'https://sci-hub.ru/10.1006/jmcc.2000.1204'},
 {'gene_id': '70',
  'symbol': 'ACTC1',
  'gene_name': 'ACTC1',
  'disease': 'Atrial septal defect 5(AD)',
  'phenotypes': 'Atrial septal defect; Ventricular septal defect',
  'reference_text': 'Monserrat, L., Hermida-Prieto, M., Fernandez, X., Rodriguez, I., Dumont, C., Cazon, L., Cuesta, M. G., Gonzalez-Juanatey, C., Peteiro, J., Alvarez, N., Penas-L

In [34]:
abl1

[{'gene_id': '25',
  'symbol': 'ABL1',
  'gene_name': 'ABL1',
  'disease': 'Congenital heart defects and skeletal malformations syndrome(AD); Leukemia, Philadelphia chromosome-positive, resistant to imatinib(SMu)',
  'phenotypes': 'Atrial septal defect; Ventricular septal defect',
  'reference_text': 'Bravo-Gil, N., Marcos, I., GonzÃ¡lez-Meneses, A., AntiÃ±olo, G., & Borrego, S. (2019). Expanding the clinical and mutational spectrum of germline ABL1 mutations-associated syndrome. Medicine, 98(10), e14782. https://doi.org/10.1097/md.0000000000014782 DOI : 10.1097/MD.0000000000014782 PMID : 30855488',
  'doi_link': 'https://doi.org/10.1097/MD.0000000000014782',
  'scihub_link': 'https://sci-hub.ru/10.1097/MD.0000000000014782'},
 {'gene_id': '25',
  'symbol': 'ABL1',
  'gene_name': 'ABL1',
  'disease': 'Congenital heart defects and skeletal malformations syndrome(AD); Leukemia, Philadelphia chromosome-positive, resistant to imatinib(SMu)',
  'phenotypes': 'Atrial septal defect; Ventricula

In [38]:
def get_gene_metadata_with_merged_references(gene_id, symbol):
    """Extract gene metadata and merge all selected references into one dictionary."""
    url = f"{BASE_URL}/gene/{gene_id}"
    res = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")

    gene_name = soup.find("h1").text.strip().split()[0]
    disease = ""
    phenotypes = []

    # --- Extract gene info ---
    rows = soup.find_all("div", class_="row")
    for row in rows:
        label_div = row.find("div", class_="col-md-3")
        value_div = row.find("div", class_="col-md-9")
        if not label_div or not value_div:
            continue
        label = label_div.get_text(strip=True)
        if label == "Disease":
            items = value_div.find_all("li")
            disease = "; ".join(li.get_text(strip=True) for li in items) if items else value_div.get_text(strip=True)
        elif label == "CHD Phenotype":
            items = value_div.find_all("li")
            phenotypes = [li.get_text(strip=True) for li in items]

    # --- Extract and merge references ---
    reference_texts = []
    doi_links = []
    scihub_links = []

    ref_header = soup.find("h3", string=lambda x: x and "Selected References" in x)
    if ref_header:
        ref_list = ref_header.find_next("ol")
        if ref_list:
            for li in ref_list.find_all("li"):
                citation = li.get_text(separator=" ", strip=True)
                doi = ""
                for s in li.find_all("strong"):
                    if "DOI" in s.text:
                        a_tag = s.find_next("a")
                        if a_tag and "doi.org" in a_tag["href"]:
                            doi = a_tag["href"].split("doi.org/")[-1]
                        elif a_tag:
                            doi = a_tag.text.strip()
                        break
                reference_texts.append(citation)
                if doi:
                    doi_links.append(f"https://doi.org/{doi}")
                    scihub_links.append(f"https://sci-hub.ru/{doi}")

    return {
        "gene_id": gene_id,
        "symbol": symbol,
        "gene_name": gene_name,
        "disease": disease,
        "phenotypes": "; ".join(phenotypes),
        "reference_texts": reference_texts,
        "doi_links": doi_links,
        "scihub_links": scihub_links
    }


In [39]:
actc1 = get_gene_metadata_with_merged_references('70','ACTC1')

In [40]:
actc1

{'gene_id': '70',
 'symbol': 'ACTC1',
 'gene_name': 'ACTC1',
 'disease': 'Atrial septal defect 5(AD)',
 'phenotypes': 'Atrial septal defect; Ventricular septal defect',
 'reference_texts': ['Olson, T. M., Doan, T. P., Kishimoto, N. Y., Whitby, F. G., Ackerman, M. J., & Fananapazir, L. (2000). Inherited and de novo Mutations in the Cardiac Actin Gene Cause Hypertrophic Cardiomyopathy. Journal of Molecular and Cellular Cardiology, 32(9), 1687â€“1694. https://doi.org/10.1006/jmcc.2000.1204 DOI : 10.1006/jmcc.2000.1204',
  'Monserrat, L., Hermida-Prieto, M., Fernandez, X., Rodriguez, I., Dumont, C., Cazon, L., Cuesta, M. G., Gonzalez-Juanatey, C., Peteiro, J., Alvarez, N., Penas-Lado, M., & Castro-Beiras, A. (2007). Mutation in the alpha-cardiac actin gene associated with apical hypertrophic cardiomyopathy, left ventricular non-compaction, and septal defects. European Heart Journal, 28(16), 1953â€“1961. https://doi.org/10.1093/eurheartj/ehm239 DOI : 10.1093/eurheartj/ehm239 PMID : 17611253

In [53]:
analyzed_gene_list_c = [
'PRKACA',
'SMG9',
'KANSL1',
'SPRED2',
'CHD7',
'CFC1',
'HRAS',
'DOCK6',
'NOTCH2',
'KYNU',
'JAG1',
'CRELD1',
'FOXH1',
'PLXND1',
'ZFPM2',
'KMT2A',
'NPHP3',
'MED13L',
'SMAD4',
'SON',
'ZEB2',
'ANKRD11',
'PIGV']